In [1]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud, STOPWORDS

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from pprint import pprint

import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
movies = pd.read_csv("data/movies_3.csv", sep=",")
directors = pd.read_csv("data/directors_crew.csv", sep=",")

In [3]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45333 entries, 0 to 45332
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_movie           45333 non-null  int64  
 1   title              45333 non-null  object 
 2   tagline            20381 non-null  object 
 3   overview           44392 non-null  object 
 4   runtime            45087 non-null  float64
 5   release_date       45333 non-null  object 
 6   release_year       45333 non-null  int64  
 7   status             45253 non-null  object 
 8   original_language  45322 non-null  object 
 9   popularity         45333 non-null  float64
 10  vote_average       45333 non-null  int64  
 11  vote_count         45333 non-null  int64  
 12  budget             45333 non-null  int64  
 13  revenue            45333 non-null  int64  
 14  return             45333 non-null  float64
 15  name_collection    4376 non-null   object 
 16  id_collection      448

In [4]:
movies.describe()

,id_movie,runtime,release_year,popularity,vote_average,vote_count,budget,revenue,return,id_collection
count,45333.000000,45087.000000,45333.000000,45333.000000,45333.000000,45333.000000,4.533300e+04,4.533300e+04,4.533300e+04,4483.000000
mean,108058.249642,94.175882,1991.882536,2.925689,5.250369,110.149207,4.230989e+06,1.123304e+07,1.578364e+02,184212.998885
std,112196.879634,38.351846,24.053205,6.011321,1.858919,491.967458,1.744452e+07,6.441813e+07,1.313633e+04,141638.033338
min,2.000000,0.000000,1874.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,10.000000
25%,26398.000000,85.000000,1978.000000,0.388631,5.000000,3.000000,0.000000e+00,0.000000e+00,0.000000e+00,86027.500000
50%,59853.000000,95.000000,2001.000000,1.129891,6.000000,10.000000,0.000000e+00,0.000000e+00,0.000000e+00,141615.000000
75%,156700.000000,107.000000,2010.000000,3.687905,6.000000,34.000000,0.000000e+00,0.000000e+00,0.000000e+00,294179.500000
max,469172.000000,1256.000000,2020.000000,547.488298,10.000000,14075.000000,3.800000e+08,2.787965e+09,1.916667e+06,480160.000000


Podemos ver que mas del 75% de las peliculas tienen un return = 0 por lo que utilizaremos este dato para filtrar las peliculas con las que entrenaremos nuestro modelo.

In [5]:
movies = movies[movies['vote_count'] > 2000]
movies.shape

(512, 17)

In [6]:
sist_recomendacion_df = (
     pd.merge(
         movies[['title','overview','id_movie']],
         directors,
         on = 'id_movie',
         how='inner')
     )

# sist_recomendacion_df

In [7]:
sist_recomendacion_df = sist_recomendacion_df.groupby(['title','overview'], as_index=False)['name'].agg(' '.join)
sist_recomendacion_df

,title,overview,name
0,(500) Days Of Summer,"Tom (Joseph Gordon-Levitt), greeting-card writ...",Mychael Danna Hope Hanafin Steven J. Wolfe Mas...
1,10 Cloverfield Lane,"After a car accident, Michelle awakens to find...",Monika Mikkelsen J.J. Abrams Matthew W. Mungle...
2,12 Angry Men,The defense and the prosecution have rested an...,Sidney Lumet Reginald Rose Henry Fonda Kenyon ...
3,12 Years A Slave,"In the pre-Civil War United States, Solomon No...",Hans Zimmer Brad Pitt Arnon Milchan Tessa Ross...
4,127 Hours,The true story of mountain climber Aron Ralsto...,Tessa Ross Jon Harris Anthony Dod Mantle Danny...
...,...,...,...
507,X-Men: The Last Stand,"When a cure is found to treat mutations, lines...",Mark Goldblatt Mark Helfrich Ralph Winter Rich...
508,X2,Professor Charles Xavier and his team of genet...,Zak Penn David Hayter Bryan Singer Michael Dou...
509,Zodiac,The true story of the investigation of 'The Zo...,Victor J. Zolfo Casey Storm David Fincher Ceán...
510,Zombieland,Columbus has made a habit of running from what...,Rhett Reese Ezra Swerdlow Gavin Polone Ryan Ka...


In [8]:
print(
    sist_recomendacion_df
    .isnull()
    .sum()
)
print (sist_recomendacion_df.shape)

title       0
overview    0
name        0
dtype: int64
(512, 3)


In [9]:
sist_recomendacion_df['title'] = sist_recomendacion_df['title'].astype('str').str.title().str.strip()
sist_recomendacion_df['overview']= sist_recomendacion_df ['overview'].astype('str')
sist_recomendacion_df['name'] = sist_recomendacion_df['name'].astype('str')

sist_recomendacion_df['text'] = sist_recomendacion_df['overview'] +  sist_recomendacion_df['name']

df = sist_recomendacion_df[['title', 'text']]

df.to_csv("data/ml_df.csv",index=False)

---

# Sistema de Recomendación

In [10]:
# Instanciamos el CV
vectorizer = CountVectorizer()
stopwords = STOPWORDS
# eliminamos las "stop words", palabras comunes no informativas
tf = TfidfVectorizer(stop_words='english')

# calculamos los features para cada ítem (texto)
tfidf_matrix = tf.fit_transform(df['text'])

# calculamos las similitudes entre todos los documentos
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
n = 5

results = {} 
for idx, row in df.iterrows():
    # guardamos los indices similares basados en la similitud coseno. Los ordenamos en modo ascendente, siendo 0 nada de similitud y 1 total
    similar_indices = cosine_similarities[idx].argsort()[:-n-2:-1] 
    # guardamos los N más cercanos
    similar_items = [(f"{df.loc[i, 'title']}") for i in similar_indices]
    results[f"{row['title']}"] = similar_items[1:]


In [11]:
pprint(results['Toy Story'])
pprint(results['Toy Story 2'])
pprint(results['Cars'])

['Toy Story 2', 'Monsters, Inc.', 'Toy Story 3', 'Finding Nemo', 'Cars']
['Monsters, Inc.', 'Toy Story', 'Finding Nemo', 'Up', 'Toy Story 3']
['Cars 2', 'Monsters, Inc.', 'Toy Story 2', 'Up', 'Finding Nemo']


-----------

In [12]:
# ML
# @app.get('/recomendacion/{titulo}')
def recomendacion(titulo:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    titulo = titulo.title().strip()

    if df['title'].str.contains(titulo).any():
        titulo = titulo.title().strip()
        lista = (results[titulo])
        data = {'titulo':titulo , 'lista recomendada': lista}
    else:
        mensaje = "La pelicula {} no se encuentra en la base de datos.".format(titulo)
        data = {'actor':[mensaje] }    
    return data

In [13]:
recomendacion('Monsters, inc. ')

{'titulo': 'Monsters, Inc.',
 'lista recomendada': ['Toy Story 2',
  'The Incredibles',
  'Finding Nemo',
  'Up',
  'Toy Story']}